In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import math
from tqdm import tqdm
import random
import urllib.request
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException


In [2]:
driver = webdriver.Chrome('/Users/heni1/chromedriver.exe')

In [3]:
#지역 식당 검색
def search(location): 
    #지역 검색
    url = 'https://www.yelp.com/'
    driver.get(url)
    time.sleep(2)

    #Find
    driver.find_element(By.ID, 'find_desc').click()
    find = driver.find_element(By.ID, 'find_desc')
    find.send_keys('Restaurants') 

    #Near
    driver.find_element(By.ID, 'dropperText_Mast').click()
    near = driver.find_element(By.ID, 'dropperText_Mast')
    near.send_keys(location) 

    driver.find_element(By.ID, 'header-search-submit').click()


In [32]:
#한 식당 범주별 사진 수집
def photo_crawler(driver, res_name):
    #여기서 식당 정보 받아오면 대
    driver.find_element(By.CLASS_NAME, 'css-1w8jvj').click()
    time.sleep(3)
    first = driver.current_url+'?tab={}'
    tab_lst = []
    for element in driver.find_elements(By.CLASS_NAME, 'tab-link_label'):
        if element.text != '':
            tab_lst.append(element.text)
    for i in tab_lst:
        if i != 'All' and i != 'Menu':
            if i == 'Inside':
                i = 'interior'
            driver.get(first.format(i.lower()))
            current_url = driver.current_url
            html = driver.page_source
            test = bs(html, 'html.parser')
            num = 1 #사진이름
            page_num = 0 #페이지번호

            #저장경로
            path = '/Users/heni1/Desktop/구하은/석사 1기/Yelp/Yelp_dataset/{}/'
            path = path.format(i.lower())
            page_num_result= test.find('div', class_ = 'page-of-pages arrange_unit arrange_unit--fill')#페이지 넘버있는 div
            last_page_num = int(page_num_result.text.split('of ')[1].split('\n')[0])# 페이지가 몇개가 있는지 

                #범주별 사진 가져오기
            for page_num in range(0, last_page_num): 
                driver.get(current_url+"&start="+str(page_num*30))
                time.sleep(3)

                pic_html = driver.page_source
                pic_test = bs(pic_html, 'html.parser')
                result = pic_test.find('div', class_ = 'media-landing_gallery photos').find_all('div', 'photo-box photo-box--interactive')
                imgUrl = [(div.find('img')['src']) for div in result]
                for j in imgUrl:
                    urllib.request.urlretrieve(j, path + res_name +'_'+ i.lower() +'_'+ str(num)+'.jpg')
                    num+=1
                page_num += 1

In [31]:
#한도시의 식당 싹 다 가능
def res_crawler(location, df):   
    new = []
    search(location)
    page = 0
    loca_url = driver.current_url #도시 검색했을 때 url
    html = driver.page_source
    test = bs(html, 'html.parser')
    loca_page_num = test.find('div', class_ = 'border-color--default__09f24__NPAKY text-align--center__09f24__fYBGO').find('span', class_ = 'css-chan6m')
    loca_page_num = int(loca_page_num.text.split('of ')[1].split('\n')[0]) #도시 검색 최대 페이지 수
    for page in range(0, loca_page_num):
        driver.get(loca_url+'&start='+str(page*10))
        time.sleep(3)
        loca_page_url = driver.current_url #
        for i in range(9, 19):
            driver.find_element_by_css_selector('#main-content > div > ul > li:nth-child('+str(i)+') > div').click()
            res_name= res_info_crawler(driver, df)
            photo_crawler(driver, res_name)
            driver.get(loca_page_url)
        page += 1 #도시 검색 페이지 더하기

In [ ]:
df =[]
res_crawler('NewYork', df)
new = pd.DataFrame(df, columns = ['name', 'rating', 'category', 'price', 'address'])
new.to_csv('restaurant.csv')

In [28]:
def res_info_crawler(driver, df): 
    html = driver.page_source
    res_cate =[]
    test = bs(html, 'html.parser')
    category_nums = len(test.find_all('span', 'css-1fdy0l5'))
    res_name = test.select('h1')[0].text
    res_rating = float(str(test.select('body > yelp-react-root > div:nth-child(1) > div.photoHeader__09f24__nPvHp.border-color--default__09f24__NPAKY > div.photo-header-content-container__09f24__jDLBB.border-color--default__09f24__NPAKY > div.photo-header-content__09f24__q7rNO.padding-r2__09f24__ByXi4.border-color--default__09f24__NPAKY > div > div > div.arrange__09f24__LDfbs.gutter-1-5__09f24__vMtpw.vertical-align-middle__09f24__zU9sE.margin-b2__09f24__CEMjT.border-color--default__09f24__NPAKY > div:nth-child(1) > span > div')[0])[17:20])
    for category_num in range(1, category_nums - 1):
            cate = test.find_all('span', 'css-1fdy0l5')[category_num].text.split(',')[0]
            res_cate.append(cate)
    res_price = test.find_all('span','css-1ir4e44')[0].text
    res_address = test.find_all('p', 'css-qyp8bo')[0].text
    df.append([res_name, res_rating, res_cate, res_price, res_address])
    return res_name
    